In [16]:
from google.cloud import storage
sc.addFile("/home/dataproc/inverted_index_gcp.py")
import inverted_index_gcp

bucket_name = 'project_ir_test'
client = storage.Client('elated-chassis-334219')
bucket = client.bucket(bucket_name)


# download index file and save it into the indexes variables
def get_index_from_storage(bucket, storage_path, index_name):
    blob = storage.Blob(f'postings_gcp/{storage_path}/{index_name}.pkl', bucket)

    with open(f'./{index_name}.pkl', "wb") as file_obj:
        blob.download_to_file(file_obj)

    return inverted_index_gcp.InvertedIndex.read_index("./", index_name)

#download bins files
def get_bins_from_storage(bucket_name, storage_path):

    os.makedirs(f'./postings_gcp/{storage_path}', exist_ok=True)
    blobs = client.list_blobs(bucket_name, prefix=f'postings_gcp/{storage_path}')

    for blob in blobs:
        if blob.name.endswith('.bin'):
            with open(f'./{blob.name}', "wb") as file_obj:
                blob.download_to_file(file_obj)


In [17]:
storage_path_body = "index_body"
body_index = get_index_from_storage(bucket, storage_path_body, 'index_body')
get_bins_from_storage(bucket_name, storage_path_body)

In [18]:
def get_posting_gen(index, bin_directory, query):
    """
    Return the generator working with posting list.
    Parameters: index: inverted index
    ----------
    """
    words, pls = zip(*index.posting_lists_iter(bin_directory, query))
    return words, pls

In [20]:
sc.addFile("/home/dataproc/rank_functions.py")
import rank_functions as rf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
words_body, pls_body = get_posting_gen(body_index, 'postings_gcp/index_body', ["python"])
vec_q = rf.generate_query_tfidf_vector(["python"], body_index)
mat_tfidf = rf.generate_document_tfidf_matrix(["python"], body_index, words_body, pls_body)
rf.cosine_similarity(mat_tfidf, vec_q, body_index)

{1857: 0.0020640929075288845,
 4800: 0.0043620403676964785,
 5213: 0.01131778619712571,
 5323: 0.003911406815107791,
 6690: 0.005034835206690497,
 6788: 0.036146683705748237,
 7729: 0.001463257918630165,
 7959: 0.005967287159785532,
 8495: 0.012972597894811233,
 9232: 0.00901161838573541,
 9457: 0.06781222369528614,
 11921: 0.010681253846311817,
 12232: 0.007863465694942187,
 12772: 0.010485476003587884,
 13149: 0.013832752583725159,
 15858: 0.006594770572491411,
 16899: 0.003733372808838533,
 18942: 0.006959669841146215,
 19390: 0.01823582571877736,
 20229: 0.02510110363735035,
 20455: 0.00479200413375199,
 20840: 0.003055098199877373,
 23298: 0.009252971543120301,
 24001: 0.021812259427609636,
 25614: 0.0023226513010634244,
 28239: 0.00488984840393334,
 28327: 0.016461961407088097,
 29831: 0.004678109188379457,
 30332: 0.009507654315406963,
 30475: 0.0031134892519576658,
 31034: 0.003099547092294159,
 31550: 0.01216606307947805,
 31664: 0.018660914713146657,
 32591: 0.002870781532029

In [70]:
query = ["migraine"]
words_body, pls_body = get_posting_gen(body_index, 'postings_gcp/index_body', query)
doc_score_top_n = rf.get_topN_score_for_query(query, body_index, words_body, pls_body, N=100)
doc = [doc for doc,score in doc_score_top_n]
migraine = [21035, 36984150, 2702930, 25045060, 24224679, 2555865, 36579642, 310429, 22352499, 11495285, 22294424, 234876, 40748148, 69893, 61962436, 62871079, 843361, 7362700, 16982268, 15712244, 5690287, 7362738, 600236, 12410589, 26584776, 3332410, 20038918, 739855, 1015919, 14201682, 24361010, 53035710, 22901459, 57672434, 4206029, 738384, 36579839, 188521, 15325435, 3602651, 40428462, 322197, 19592340, 3868233, 2385806, 2933438, 23174077, 14001660, 2425344, 288328, 21381229, 26585811, 12652799, 322210, 51078678, 621531, 685130, 11193835, 21197980, 21078348, 3108484, 692988, 31556991, 18741438, 3053003, 50977642, 55115883, 17208913, 64269900, 54077917, 36666029, 50083054, 28245491, 5692662, 18353587, 1994895, 21364162, 20208066, 38574433, 910244, 6154091, 67754025, 2132969, 61386909, 18600765, 579516],


In [71]:
len([i for i in migraine[0] if i in doc ])

82

In [73]:
len(migraine[0])

86